In [35]:
from pyspi.utils.function_utils import find_response_version
from pyspi.utils.response.spi_response_data import ResponseDataRMF
from pyspi.utils.response.spi_response import ResponseRMFGenerator
from pyspi.utils.response.spi_drm import SPIDRM
from pyspi.utils.livedets import get_live_dets

import numpy as np


In [36]:
time1 = "051105 061512"
ra, dec = 10., -40.

In [37]:
version = find_response_version(time1)
rsp_base = ResponseDataRMF.from_version(version)
dets = get_live_dets(time=time1, event_types=["single"])

Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)


In [38]:
ein = np.array(np.linspace(20,1000,4))
ebounds = np.array(np.linspace(20,1000,4))
d = 0

In [39]:
rsp1 = ResponseRMFGenerator.from_time(time1,  d,
                                            ebounds, ein,
                                            rsp_base)
sd1 = SPIDRM(rsp1, ra, dec)
print(ein)
print(sd1.matrix)

[  20.          346.66666667  673.33333333 1000.        ]
[[13.823791    2.70435285  1.79275684]
 [ 0.          8.53337789  1.70787271]
 [ 0.          0.          6.04052616]]


In [40]:
ein = np.array(np.linspace(20,1000,7))
ebounds = np.array(np.linspace(20,1000,4))


In [41]:
rsp2 = ResponseRMFGenerator.from_time(time1,  d,
                                            ebounds, ein,
                                            rsp_base)
sd2 = SPIDRM(rsp2, ra, dec)
print(ein)
print(sd2.matrix)

[  20.          183.33333333  346.66666667  510.          673.33333333
  836.66666667 1000.        ]
[[14.06299834 15.31597976  2.85573938  2.29760791  1.91975918  1.63880981]
 [ 0.          0.          8.38959361  8.25712902  1.8686086   1.15987021]
 [ 0.          0.          0.          0.          5.73556994  5.82934172]]


In [42]:
print(sd1.matrix[0,0])
print(sd2.matrix[0,:2])

13.82379100056793
[14.06299834 15.31597976]


In [43]:
# should the value from sd1 not lie inbetween the two sd2 values?